In [1]:
import requests
import pandas as pd
import numpy as np
from keys import *

# Realiza una solicitud a la API de OpenWeather para obtener el pronóstico meteorológico de la ciudad 'Girona' en España
city = "Barcelona"
country = "ES"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')

In [2]:
data = response.json()

In [3]:
# Obtiene la lista de previsiones meteorológicas del diccionario 'data'
forecast_list = data.get('list', [])

In [4]:
# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Barcelona,ES"
})

print(df.head())

      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-10-30 03:00:00        16.40        74           Rain  2.73      0.17   
1  2025-10-30 06:00:00        15.01        77           Rain  2.39      0.43   
2  2025-10-30 09:00:00        15.41        71         Clouds  2.76       NaN   
3  2025-10-30 12:00:00        21.26        61         Clouds  6.40       NaN   
4  2025-10-30 15:00:00        20.36        60          Clear  7.28       NaN   

   snow municipality_iso_country  
0   NaN             Barcelona,ES  
1   NaN             Barcelona,ES  
2   NaN             Barcelona,ES  
3   NaN             Barcelona,ES  
4   NaN             Barcelona,ES  


In [ ]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = ""
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [6]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40